# Environment Setup for Soccer Tracking Pipeline

This notebook helps you set up the environment for running the soccer tracking pipeline on Google Colab with remote VM access via VS Code SSH.

## 1. Install Dependencies

In [ ]:
# Install PyTorch with CUDA support (automatically detects available CUDA version)
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Install computer vision and tracking libraries
!pip install ultralytics boxmot opencv-python numpy scipy

In [ ]:
# Install evaluation library
!pip install -q git+https://github.com/JonathonLuiten/TrackEval.git

In [ ]:
# Install additional utilities
!pip install tqdm matplotlib pandas pyyaml

## 2. Mount Google Drive (for data access)

In [ ]:
# Mount Google Drive for dataset access
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
    
    # Check if soccer data exists
    import os
    gdrive_path = "/content/drive/MyDrive/SOCCER_DATA"
    if os.path.exists(gdrive_path):
        print(f"Soccer data found at: {gdrive_path}")
        print("Available datasets:")
        for item in os.listdir(gdrive_path):
            print(f"  - {item}")
    else:
        print(f"No soccer data found at: {gdrive_path}")
        print("Please upload your dataset to Google Drive first.")
        
except ImportError:
    print("Not running in Google Colab. Skipping drive mount.")
except Exception as e:
    print(f"Error mounting drive: {e}")

## 3. Set Up the Project

In [ ]:
# Clone the repository if not already present
import os

if not os.path.exists('/content/soccer-tracking-pipeline'):
    !git clone https://github.com/yourusername/soccer-tracking-pipeline.git
    
%cd /content/soccer-tracking-pipeline

# Install project dependencies
!pip install -r requirements.txt

## 4. Verify Installation and Hardware

In [ ]:
# Check hardware and installations
import torch
import cv2
import numpy as np
from ultralytics import YOLO

print("=== Hardware Information ===")
print(f"Python version: {torch.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
        print(f"  GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")

print(f"\n=== Library Versions ===")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

# Test YOLO
try:
    model = YOLO('yolov8n.pt')
    print(f"YOLO model loaded successfully")
except Exception as e:
    print(f"Error loading YOLO: {e}")

# Test BotSort
try:
    from boxmot import BotSORT
    print(f"BotSORT available")
except ImportError:
    print(f"BotSORT not available - install with: pip install boxmot")

print("\n✓ All core libraries loaded successfully!")

## 5. Download Sample Data and Models

In [ ]:
# Download sample data and models
!python scripts/download_models.py --all

## 6. Set Up SSH for Remote Development (Optional)

In [ ]:
# Install and configure SSH for VS Code remote development
!pip install colab-ssh -q

print("SSH setup ready. To connect VS Code remotely, run:")
print("")
print("from colab_ssh import launch_ssh_cloudflared")
print("launch_ssh_cloudflared(password='your_secure_password')")
print("")
print("Then follow the instructions to connect VS Code.")

In [ ]:
# Uncomment and run this cell to start SSH server
# from colab_ssh import launch_ssh_cloudflared
# launch_ssh_cloudflared(password="your_secure_password_here")

## 7. Copy Dataset from Google Drive to Local Storage

In [ ]:
# Copy dataset from Google Drive to local SSD for faster access
import shutil
from pathlib import Path

gdrive_dataset = "/content/drive/MyDrive/SOCCER_DATA/deepsort_dataset_train"
local_dataset = "/content/soccer_dataset"

if os.path.exists(gdrive_dataset):
    print(f"Copying dataset from Google Drive to local SSD...")
    print(f"Source: {gdrive_dataset}")
    print(f"Destination: {local_dataset}")
    
    if os.path.exists(local_dataset):
        print("Local dataset already exists. Skipping copy.")
    else:
        shutil.copytree(gdrive_dataset, local_dataset)
        print("Dataset copied successfully!")
    
    # Verify dataset structure
    print("\nDataset structure:")
    for root, dirs, files in os.walk(local_dataset):
        level = root.replace(local_dataset, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Show first 5 files only
            print(f"{subindent}{file}")
        if len(files) > 5:
            print(f"{subindent}... and {len(files) - 5} more files")
else:
    print(f"Dataset not found at: {gdrive_dataset}")
    print("Please upload your dataset to Google Drive first.")

## 8. Test the Pipeline

In [ ]:
# Quick test of the tracking pipeline
from src.tracking import YOLOTracker
from src.data import VideoGenerator
import cv2

# Test with sample video
sample_video = "data/sample_videos/people.mp4"

if os.path.exists(sample_video):
    print("Testing YOLO tracker on sample video...")
    
    # Initialize tracker
    tracker = YOLOTracker(device='auto')
    
    # Process a few frames
    cap = cv2.VideoCapture(sample_video)
    frame_count = 0
    
    while frame_count < 10:  # Process first 10 frames
        ret, frame = cap.read()
        if not ret:
            break
        
        tracks = tracker.update(None, frame)
        print(f"Frame {frame_count}: {len(tracks)} tracks detected")
        frame_count += 1
    
    cap.release()
    print("✓ Pipeline test completed successfully!")
else:
    print(f"Sample video not found at: {sample_video}")
    print("Run the download script first.")

## Next Steps

Your environment is now set up! You can:

1. **Run the simple tracking demo**: Open `02_simple_tracking_demo.ipynb`
2. **Process soccer datasets**: Open `03_soccer_tracking_pipeline.ipynb`
3. **Evaluate results**: Open `04_evaluation_analysis.ipynb`
4. **Use command-line scripts**:
   - Track videos: `python scripts/run_tracking.py --help`
   - Evaluate results: `python scripts/evaluate_results.py --help`

### For Remote Development:
- Connect VS Code using the SSH tunnel created above
- Access files on the VM's SSD for fast processing
- Leverage GPU acceleration automatically

### Performance Tips:
- Use the local SSD (`/content/`) for active datasets
- Keep original data on Google Drive for backup
- Monitor GPU memory usage with `nvidia-smi`